# <p style="text-align: center;">MIS 285N: Big Data and Distributed Programming</p>
# <p style="text-align: center;">Project - 1 : Apache Spark</p>
## <p style="text-align: center;">Instructor: Dr. Ramesh Yerraballi</p>
## <p style="text-align: center;">Due: Tuesday, September 14th submitted via Canvas by 11:59 pm</p>

Your work should be written in a **Jupyter notebook**.   

Also, please make sure your code runs in your notebook before submitting.

**Note:**

This project is based on Map-Reduce Framework. In these you will get to work with Spark and will get to know how 
does spark work, what functionalities does spark provide, what does map-reduce framework do and why is it useful. 

In this project you will be implementing a basic song recommender system. You will be given a dataset where there are multiple csv files. These csv files have data corresponding to song play count and song information.

The data you would be using will be provided in a zip file along with this notebook. The __msd.zip__ archive contains:
1. **'kaggle_visible_evaluation_triplets.txt'**. We will be using the visible part of the testing data to understand the working on Apache Spark.  The user's listening history is provided as: (user, song, play count).  
2. In **'kaggle_songs.txt'** file, each song is marked using an index for easier representation of songs.  
3. And **'kaggle_users.txt'** file is the canonical list of user identifiers.
4. Take **'MSDChallengeGettingstarted.pdf'** as your reference.



### **What to turn in?**  

A zip folder which will have:
1. Jupyter Notebook
2. A brief report in PDF format on what features you used for recommendation. And a brief explanation of flow of your code. For example,  what RDD does what or, why it was created.
3. datasets folder with the csv files you are using in your notebook.
4. Notebook should use relative path to the csv files in datasets folder.
5. Name of the zip folder - `<your_name>_<your_partner_name>.zip`

This project consists of 4 questions:  

1. Create an RDD with _kaggle_visible_evaluation_triplets.txt_ and replace the song name with the song index from _kaggle_songs.txt_. Identify the number of songs that do not have any rating. 
2. Generate song ratings based on the song play count as a normalized score between 0 and 1. 
3. Identify the popular song based on this rating and recommend songs to user, given user id based on the algorithm used in Movie recommender system from class. 
4. Using Cosine similarity function, identify pair-wise similarity between each pair of users and generate the top 5 most similar users without an overlap in users. 

The above list is the higer level idea about the questions. 

In [1]:
### Starter code ####
import findspark
findspark.init('/opt/homebrew/Cellar/apache-spark/3.1.2/libexec')
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster("local[*]").setAppName("Songs")
sc = SparkContext(conf = conf)
#### These lines are to tell jupyter where to find Apache Spark ####

In [2]:
#sc.stop()

In [2]:
## Read triplet file into RDD
triplet_rdd = sc.textFile(r"kaggle_visible_evaluation_triplets.txt") \
    .map(lambda line: line.split("\t")) 


 

## Step 1: 
Replace song name with song index and identify the number of songs without user history

In [3]:
# Creating a Song RDD by getting the data from kaggle_songs.txt. The seperator is a " " blank space because that is how data is separated in the text file.
song_rdd = sc.textFile(r"kaggle_songs.txt") \
    .map(lambda line: line.split(" ")) 



In [4]:
# Creating a dictionary from the Song RDD
song_dict = song_rdd.collectAsMap()


# Applying a function to replace all song names in Triplet RDD with the song index using dictionary created above
triplet_rdd = triplet_rdd.map(lambda x: (x[0],song_dict[x[1]],int(x[2])
                                        ))


In [5]:

triplet_rdd.take(5)

[('fd50c4007b68a3737fe052d5a4f78ce8aa117f3d', '25150', 1),
 ('fd50c4007b68a3737fe052d5a4f78ce8aa117f3d', '68212', 1),
 ('fd50c4007b68a3737fe052d5a4f78ce8aa117f3d', '87433', 1),
 ('fd50c4007b68a3737fe052d5a4f78ce8aa117f3d', '123630', 1),
 ('fd50c4007b68a3737fe052d5a4f78ce8aa117f3d', '58821', 1)]

In [6]:
# Creating a list of all song indexes in the Triplet RDD
ratings_list = triplet_rdd.map(lambda x: x[1]).distinct().collect()

# Creating a list of all song indexes in the Song RDD
songs_list = song_rdd.map(lambda x: x[1]).collect()

# Created a "songs without rating" list by getting all song indexes that are present in Song RDD but not in Triplet RDD
songs_without_rating = list(set(songs_list) - set(ratings_list))
len(songs_without_rating)

223007

## Step 2:
Generate song ratings based on the play_count. For example, if (song_1, 5; song_2, 10; song_3, 5) i.e., song_1 is played 5 times, song_2 is played 10 times and song_3 is played 5 times, the normalized rating score should be 0.25, 0.5 and 0.25 respectively. 
Similarly, generate the rating for all the songs. You may notice that based on all songs, the rating is almost always very low. So, think of the best way to convert song count to ratings. (Hint: Try generating ratings based on each user's song play history)

In [37]:
from collections import defaultdict
# reduce triplet_rdd into a tuple of song index and play count - song index is key, play count is value
# use reduceby to get song count by adding play count per song index (which is the key)
# Note: reduceby uses two different variables but really the variables stand for the same - adding counts in pairs.
song_count_rdd = triplet_rdd.map(lambda x: (x[1],x[2])).reduceByKey(lambda x, y: x + y)
 

#get the number of songs played per user  
song_user_count_rdd = triplet_rdd.map(lambda x: (x[0],x[2])).reduceByKey(lambda x, y: x + y)
user_and_songcount_rdd = song_user_count_rdd.map(lambda x: (x[0],x[1],1)) 
 
 
#get the song list played per user   
list_of_user_songs_rdd = triplet_rdd.map(lambda x: (x[0],x[1].split())).reduceByKey(lambda x, y: x + y)
user_and_songlist_rdd = list_of_user_songs_rdd.map(lambda x: (x[0],x[1]))
 
user_data_join = user_and_songcount_rdd.join(triplet_rdd)
find_avg_per_user = user_data_join.map(lambda x: (x[0],x[1][1],1/x[1][0])) 
print(find_avg_per_user.take(5))

sum_avg_per_song = find_avg_per_user.map(lambda x: (x[1],x[2])).reduceByKey(lambda x, y: x + y)
print(sum_avg_per_song.take(5))


#find number of users listened to a particular song
# multiply sum_avg_per_song with number of users.


[('a7db6535e9e7f76152af448875851c2174d36fd1', (10, '24495')), ('a7db6535e9e7f76152af448875851c2174d36fd1', (10, '84076')), ('a7db6535e9e7f76152af448875851c2174d36fd1', (10, '19159')), ('a7db6535e9e7f76152af448875851c2174d36fd1', (10, '347550')), ('a7db6535e9e7f76152af448875851c2174d36fd1', (10, '281233'))]
[('a7db6535e9e7f76152af448875851c2174d36fd1', '24495', 0.1), ('a7db6535e9e7f76152af448875851c2174d36fd1', '84076', 0.1), ('a7db6535e9e7f76152af448875851c2174d36fd1', '19159', 0.1), ('a7db6535e9e7f76152af448875851c2174d36fd1', '347550', 0.1), ('a7db6535e9e7f76152af448875851c2174d36fd1', '281233', 0.1)]
[('89579', 2.377832862438824), ('374773', 0.9240047229954816), ('234204', 5.476121407418219), ('6743', 1.8158141199421463), ('324001', 0.24995917034133594)]


## Step 3: 
For a given user_id (choose one by yourselves), rating, recommend 5 other songs from the list. One way to do this is based on another user who liked the same song liked by this user with rating more than the given rating and recommend the 5 songs based on the matched user's rating. 

## Step 4: 
1. Compute cosine similarity between all pairs of users. 
2. Sort the similarity score and print the top-5 similar users. 
3. If the top-5 user set has an user appearing more than once, ignore that pair and take the next best pair from the sorted list. 
4. For a given user_id, identify the top-5 similar users and hence song recommendations from other user's list. 